### Linear keras models

In [37]:
import numpy as np

In [10]:
import keras

In [32]:
x = np.random.random(size=(20000, 2))
y = np.dot(x, [3, 7]) + 5

In [33]:
x[:5], y[:5]

(array([[ 0.9338,  0.5014],
        [ 0.5133,  0.0061],
        [ 0.456 ,  0.5847],
        [ 0.0713,  0.5601],
        [ 0.1803,  0.7257]]),
 array([ 11.3113,   6.5826,  10.4611,   9.1346,  10.6209]))

In [5]:
main_layer = keras.layers.core.Dense(output_dim=1, input_shape=(2,))

lin_model = keras.models.Sequential()
lin_model.add(main_layer)

In [6]:
lin_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_1 (Dense)                  (None, 1)             3           dense_input_1[0][0]              
Total params: 3
____________________________________________________________________________________________________


In [7]:
lin_model.compile(optimizer=keras.optimizers.SGD(lr=0.1), 
                  loss='mse')

In [8]:
# error before fitting
lin_model.evaluate(x,y)

16864/20000 [========================>.....] - ETA: 0s

113.65839571533203

In [9]:
lin_model.fit(x, y, nb_epoch=20, batch_size=200, verbose=1);

Epoch 1/20
20000/20000 [==============================] - 0s - loss: 2.5068     
Epoch 2/20
20000/20000 [==============================] - 0s - loss: 0.0284     
Epoch 3/20
20000/20000 [==============================] - 0s - loss: 0.0030     
Epoch 4/20
20000/20000 [==============================] - 0s - loss: 3.4179e-04     
Epoch 5/20
20000/20000 [==============================] - 0s - loss: 3.9328e-05     
Epoch 6/20
20000/20000 [==============================] - 0s - loss: 4.5610e-06     
Epoch 7/20
20000/20000 [==============================] - 0s - loss: 5.2826e-07     
Epoch 8/20
20000/20000 [==============================] - 0s - loss: 6.1372e-08     
Epoch 9/20
20000/20000 [==============================] - 0s - loss: 7.1425e-09     
Epoch 10/20
20000/20000 [==============================] - ETA: 0s - loss: 9.6197e-1 - 0s - loss: 8.2777e-10     
Epoch 11/20
20000/20000 [==============================] - 0s - loss: 9.0148e-11     
Epoch 12/20
20000/20000 [======================

In [10]:
# error after fitting
lin_model.evaluate(x,y)

19648/20000 [============================>.] - ETA: 0s

1.2407838312356035e-11

In [11]:
lin_model.get_weights()

[array([[ 2.99999499],
        [ 6.99998903]], dtype=float32), array([ 5.00000858], dtype=float32)]

## Using vgg16 prediction vectors directly

In [12]:
import bcolz

In [18]:
def save_array(fname, arr):
    c = bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
    
def load_array(fname):
    return bcolz.open(fname)[:]

In [3]:
import vgg16
import utils

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [1]:
path = 'data/dogscats/'

In [2]:
batch_size = 64

In [4]:
import vgg16
vgg = vgg16.Vgg16()

In [24]:
train_batches = utils.get_batches(path+'train', shuffle=False, batch_size=1)
valid_batches = utils.get_batches(path+'valid', shuffle=False, batch_size=1)

Found 200 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [14]:
train_data = utils.get_data(path+'train')
valid_data = utils.get_data(path+'valid')

Found 23000 images belonging to 2 classes.


KeyboardInterrupt: 

In [28]:
train_labels = utils.onehot(train_batches.classes)
valid_labels = utils.onehot(valid_batches.classes)

In [29]:
train_features = vgg.model.predict(train_data, batch_size=batch_size)
valid_features = vgg.model.predict(valid_data, batch_size=batch_size)

In [44]:
main_layer = keras.layers.core.Dense(output_dim=2, input_shape=(1000,), activation='softmax')

lin_model = keras.models.Sequential([main_layer])

lin_model.compile(optimizer=keras.optimizers.RMSprop(lr=0.1),
                 loss='categorical_crossentropy',
                 metrics=['accuracy']
                 )

In [45]:
lin_model.fit(x=train_features, 
              y=train_labels, 
              nb_epoch=5, 
              batch_size=batch_size,
              validation_data=(valid_features, valid_labels),
             );

Train on 200 samples, validate on 200 samples
Epoch 1/5
200/200 [==============================] - 0s - loss: 0.5395 - acc: 0.7100 - val_loss: 0.3486 - val_acc: 0.9150
Epoch 2/5
200/200 [==============================] - 0s - loss: 0.2782 - acc: 0.9550 - val_loss: 0.2663 - val_acc: 0.9300
Epoch 3/5
200/200 [==============================] - 0s - loss: 0.1953 - acc: 0.9600 - val_loss: 0.2111 - val_acc: 0.9400
Epoch 4/5
200/200 [==============================] - 0s - loss: 0.1540 - acc: 0.9900 - val_loss: 0.1949 - val_acc: 0.9400
Epoch 5/5
200/200 [==============================] - 0s - loss: 0.1248 - acc: 0.9850 - val_loss: 0.1707 - val_acc: 0.9550


In [46]:
lin_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_12 (Dense)                 (None, 2)             2002        dense_input_4[0][0]              
Total params: 2002
____________________________________________________________________________________________________


In [50]:
preds = lin_model.predict_classes(valid_features, batch_size=batch_size)

 64/200 [========>.....................] - ETA: 0s

In [49]:
probs = lin_model.predict_proba(valid_features, batch_size=batch_size)[:, 0]

 64/200 [========>.....................] - ETA: 0s

In [58]:
np.stack([preds, probs], ).transpose()

array([[ 0.    ,  0.9489],
       [ 1.    ,  0.0817],
       [ 0.    ,  0.8289],
       [ 0.    ,  0.9419],
       [ 0.    ,  0.8382],
       [ 0.    ,  0.5058],
       [ 0.    ,  0.7648],
       [ 0.    ,  0.9459],
       [ 0.    ,  0.9544],
       [ 0.    ,  0.823 ],
       [ 0.    ,  0.6496],
       [ 0.    ,  0.7894],
       [ 0.    ,  0.9557],
       [ 0.    ,  0.9417],
       [ 0.    ,  0.9509],
       [ 0.    ,  0.9372],
       [ 0.    ,  0.9495],
       [ 1.    ,  0.1732],
       [ 0.    ,  0.9338],
       [ 0.    ,  0.5521],
       [ 0.    ,  0.9076],
       [ 1.    ,  0.1928],
       [ 0.    ,  0.9386],
       [ 0.    ,  0.9462],
       [ 0.    ,  0.915 ],
       [ 0.    ,  0.9457],
       [ 0.    ,  0.9464],
       [ 0.    ,  0.7793],
       [ 0.    ,  0.8704],
       [ 0.    ,  0.9352],
       [ 0.    ,  0.9261],
       [ 0.    ,  0.9185],
       [ 0.    ,  0.953 ],
       [ 0.    ,  0.9407],
       [ 0.    ,  0.9391],
       [ 0.    ,  0.9241],
       [ 0.    ,  0.9339],
 

## Finetuning vgg16 for cats v dogs

In [5]:
vgg.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [6]:
vgg.model.pop()

In [7]:
vgg.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [8]:
for layer in vgg.model.layers:
    layer.trainable = False

In [11]:
vgg.model.add(keras.layers.core.Dense(output_dim=2, activation='softmax'))

In [23]:
opt = keras.optimizers.RMSprop(lr=0.1)

vgg.model.compile(optimizer=opt,
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

In [18]:

train_batches = utils.get_batches(path+'train',
                              batch_size=batch_size,
                              shuffle=True,
                              )
valid_batches = utils.get_batches(path+'valid',
                              batch_size=batch_size,
                              shuffle=True,
                              )

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [20]:
vgg.model.fit_generator(train_batches,
                       samples_per_epoch=train_batches.N,
                       nb_epoch=1,
                       validation_data=valid_batches,
                       nb_val_samples=valid_batches.N,
                       )

Epoch 1/1
23000/23000 [==============================] - 590s - loss: 0.6605 - acc: 0.9577 - val_loss: 0.3304 - val_acc: 0.9790


In [ ]:
vgg.model.save_weights('models/dogscats/finetune1.h5')

In [24]:
vgg.model.load_weights('models/dogscats/finetune1.h5')

## More finely tuned finetuning of all Dense layers

In [26]:
# find first dense layer
for idx, layer in enumerate(vgg.model.layers):
    if type(layer) == keras.layers.core.Dense:
        break
        
for layer in vgg.model.layers[idx:]:
    layer.trainable = True

In [28]:
keras.backend.set_value(opt.lr, 0.01)

In [29]:
vgg.model.fit_generator(train_batches,
                       samples_per_epoch=train_batches.N,
                       nb_epoch=3,
                       validation_data=valid_batches,
                       nb_val_samples=valid_batches.N,
                       )

Epoch 1/3
23000/23000 [==============================] - 591s - loss: 0.4170 - acc: 0.9733 - val_loss: 0.1864 - val_acc: 0.9880
Epoch 2/3
23000/23000 [==============================] - 589s - loss: 0.4040 - acc: 0.9743 - val_loss: 0.2707 - val_acc: 0.9830
Epoch 3/3
23000/23000 [==============================] - 590s - loss: 0.3886 - acc: 0.9750 - val_loss: 0.2363 - val_acc: 0.9845


In [30]:
vgg.model.save_weights('models/dogscats/finetune2.h5')

## Finely Tune some convolutions too

In [31]:
for layer in vgg.model.layers[12:]:
    layer.trainable = True

keras.backend.set_value(opt.lr, 0.001)

In [32]:
vgg.model.fit_generator(train_batches,
                       samples_per_epoch=train_batches.N,
                       nb_epoch=4,
                       validation_data=valid_batches,
                       nb_val_samples=valid_batches.N,
                       )

Epoch 1/4
23000/23000 [==============================] - 590s - loss: 0.3835 - acc: 0.9756 - val_loss: 0.2211 - val_acc: 0.9860
Epoch 2/4
23000/23000 [==============================] - 590s - loss: 0.3558 - acc: 0.9773 - val_loss: 0.2306 - val_acc: 0.9855
Epoch 3/4
23000/23000 [==============================] - 590s - loss: 0.3574 - acc: 0.9770 - val_loss: 0.2244 - val_acc: 0.9860
Epoch 4/4
23000/23000 [==============================] - 590s - loss: 0.3800 - acc: 0.9760 - val_loss: 0.1942 - val_acc: 0.9875


In [33]:
vgg.model.save_weights('models/dogscats/finetune3.h5')

## Make submission

In [42]:
import pandas as pd

In [35]:
test_batches, predictions = vgg.test(path+'test')

Found 12500 images belonging to 1 classes.


In [38]:
# check for extreme predictions
if np.any(predictions == 1) or np.any(predictions == 0):
    predictions = np.clip(predictions[:,1], 0.01, 0.99)

In [39]:
submission_rows = [ (int(filename.replace('unknown/', '').replace('.jpg', '')),
                     predic
                     )
        for predic, filename in zip(predictions, test_batches.filenames)
]

In [43]:
df = pd.DataFrame(submission_rows)
df.columns = ['id', 'label']
df.head()

,id,label
0,9292,0.01
1,12026,0.01
2,9688,0.01
3,4392,0.01
4,779,0.99


In [44]:
df.to_csv('data/dogscats/submission.csv', index=False)